## Importing Libraries

In [1]:
# pytorch
import torch

# huggingface
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

## Hugging Face

In [2]:
model_name = "Waktaverse-Llama-3-KO-8B-Instruct"  # ADD YOUR MODEL NAME HERE
username = "PathFinderKR"  # ADD YOUR USERNAME HERE
repo_id = f"{username}/{model_name}"  # repository id

## Device

In [3]:
# Device setup
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


## Merge

In [4]:
korean_tokenizer_id = "saltlux/Ko-Llama3-Luxia-8B"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [5]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(korean_tokenizer_id)
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
# Resize token embeddings
base_model.resize_token_embeddings(len(tokenizer) + 1)
# Merge and unload
model = PeftModel.from_pretrained(base_model, model_name)
model = model.merge_and_unload()

## Upload

In [8]:
tokenizer.push_to_hub(
    repo_id=repo_id,
    use_temp_dir=False
)
model.push_to_hub(
    repo_id= repo_id,
    use_temp_dir=False
)

README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.55G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/PathFinderKR/Waktaverse-Llama-3-KO-8B-Instruct/commit/14db81a81b7dd0cd9385cc0069e8b9f908348909', commit_message='Upload LlamaForCausalLM', commit_description='', oid='14db81a81b7dd0cd9385cc0069e8b9f908348909', pr_url=None, pr_revision=None, pr_num=None)